In [1]:
## torch.autograd라는 자동 미분 엔진이 내장되어 있음. 모든 계산 그래프에 대한 변화도의 자동 계산을 지원함
import torch

x = torch.ones(5)
y = torch.zeros(3)
w = torch.randn(5, 3, requires_grad = True) ## 학습해야하는 파라미터들에 대해서 손실 함수의 변화도 계산 위한 절차
b = torch.randn(3, requires_grad = True) ## 마찬가지
z = torch.matmul(x, w) + b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

In [2]:
## grad_fn은 순전파랑 역전파 계산하는 방법을 알고 있음
print(f'Gradient function for z = {z.grad_fn}')
print(f'Gradient function for loss = {loss.grad_fn}')

Gradient function for z = <AddBackward0 object at 0x7fd8fb7ae7d0>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward0 object at 0x7fd8fb7afd30>


In [4]:
## 가중치 최적화 위해 매개변수에 대한 손실함수의 도함수를 계산해야함.
## 각 매개변수 별로 loss에 대해서 w와 b의 편미분이 필요함
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.2802, 0.1434, 0.2431],
        [0.2802, 0.1434, 0.2431],
        [0.2802, 0.1434, 0.2431],
        [0.2802, 0.1434, 0.2431],
        [0.2802, 0.1434, 0.2431]])
tensor([0.2802, 0.1434, 0.2431])


In [5]:
## requires_grad = True의 경우에는 연산 기록을 추적하고 변화도 계산을 지원함.
## 만약, 순전파 연산만 필요하다면, 이러한 추적이나 지원이 필요하지 않을 수도 있음. => torch.no_grad()

z = torch.matmul(x, w) + b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w) + b
print(z.requires_grad)

True
False


In [7]:
z = torch.matmul(x, w) + b
z_det = z.detach() ## 역전파 안함
print(z.requires_grad)
print(z_det.requires_grad)

True
False


In [ ]:
#### 변화도 추적을 멈춰야 하는 경우 ####
## 신경망의 일부 매개변수를 고정된 매개변수로 표시하는 경우
## 변화도를 추적하지 않는 텐서의 연산이 더 효율적이기 때문에, 순전파 단계만 수행할 때 연산 속도가 향상됌.

In [11]:
## autograd는 데이터(텐서) 및 실행된 모든 연산들의 기록을 Function 객체로 구성된 방향성 비순환 그래프에 저장함
# 순전파 -> 요청된 연산을 수행해서 결과 텐서를 계산하고 DAG에 연산의 변화도 기능을 유지함
# 역전파 -> DAG의 뿌리(결과)에서 .backward()가 호추로딜 때 시작함
## .grad_fn으로부터 변화도 계산, 각 텐서의 .grad 속성에 계산 결과를 쌓고(accumulate), 연쇄법칙을 이용해서 모든 잎 텐서까지 전파

## backward에 매개변수가 들어가는 이유 -> 결과 자체가 행렬이기 때문에
inp = torch.eye(4, 5, requires_grad = True)
out = (inp + 1).pow(2).t()
out.backward(torch.ones_like(out), retain_graph = True) ## 여기서 여러번의 backward 계산이 필요함
print(f'First call\n{inp.grad}')
out.backward(torch.ones_like(out), retain_graph = True)
print(f'\nSecond call\n{inp.grad}')
inp.grad.zero_()
out.backward(torch.ones_like(out), retain_graph = True)
print(f'\nCall after zeroing gradients\n{inp.grad}')

## 역방향 전파 수행 시, PyTorch가 변화도를 누적해주기 때문임. 계산된 변화도의 값이 연산 그래프의 모든 노드의 grad 속성에 추가
## 따라서, 제대로 된 변화도를 계산하려면 grad 속성을 0으로 만들어 줘야 함.

First call
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])

Second call
tensor([[8., 4., 4., 4., 4.],
        [4., 8., 4., 4., 4.],
        [4., 4., 8., 4., 4.],
        [4., 4., 4., 8., 4.]])

Call after zeroing gradients
tensor([[4., 2., 2., 2., 2.],
        [2., 4., 2., 2., 2.],
        [2., 2., 4., 2., 2.],
        [2., 2., 2., 4., 2.]])
